# Ejercicio 2: Modelo de q-Coloraciones

**Estudiantes:**
- Sergio Andrés Díaz Vera (seadiazve@unal.edu.co)
- Julián Mateo Espinosa Ospina (juespinosao@unal.edu.co)

**Curso:** Cadenas de Markov y Aplicaciones (2025-II)

## Descripción

Implementación del Gibbs Sampler para el modelo de q-coloraciones propias en una rejilla K×K.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys
import os

# Agregar ruta al directorio padre para importar módulos
sys.path.append(os.path.join(os.path.dirname(os.path.abspath('')), '..'))

from src.q_coloraciones import gibbs_sampler_q_coloraciones, es_coloracion_propia, contar_colores
from src.visualizacion import visualizar_configuracion, graficar_distribucion_colores

## 1. Implementación del Gibbs Sampler para q-coloraciones

El algoritmo de Gibbs Sampler para q-coloraciones:

1. Inicializa con una coloración válida (tipo tablero de ajedrez para q≥2)
2. En cada iteración:
   - Selecciona un sitio $(i,j)$ uniformemente al azar
   - Identifica los colores prohibidos (usados por vecinos)
   - Asigna un color uniformemente de los colores disponibles

In [ ]:
# Parámetros
K = 10
q = 3
T = 10000
semilla = 42

# Generar configuración
config = gibbs_sampler_q_coloraciones(K, q, T, semilla=semilla)

print(f"Configuración generada para K={K}, q={q}, T={T}")
print(f"¿Es coloración propia?: {es_coloracion_propia(config, q)}")
print(f"\nDistribución de colores:")
conteo = contar_colores(config, q)
for color in range(q):
    print(f"  Color {color}: {conteo[color]} celdas")

In [ ]:
# Visualizar configuración
visualizar_configuracion(config, titulo=f"q-coloración: K={K}, q={q}, T={T}", 
                        colors=plt.cm.get_cmap('tab10', q))
plt.show()

## 2. Análisis de convergencia

Verificamos la evolución con diferentes valores de T.

In [ ]:
# Diferentes tiempos
T_valores = [0, 100, 1000, 10000, 100000]
K = 10
q = 3

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()
cmap = plt.cm.get_cmap('tab10', q)

for idx, T in enumerate(T_valores):
    config = gibbs_sampler_q_coloraciones(K, q, T, semilla=42)
    conteo = contar_colores(config, q)
    
    axes[idx].imshow(config, cmap=cmap, interpolation='nearest', vmin=0, vmax=q-1)
    axes[idx].set_title(f"T={T}")
    axes[idx].grid(True, alpha=0.3)

axes[-1].axis('off')
plt.tight_layout()
plt.show()

## 3. Múltiples muestras independientes

Generamos varias muestras para verificar la variabilidad y uniformidad.

In [ ]:
K = 10
q = 3
T = 10000
n_muestras = 6

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()
cmap = plt.cm.get_cmap('tab10', q)

for i in range(n_muestras):
    config = gibbs_sampler_q_coloraciones(K, q, T, semilla=i)
    conteo = contar_colores(config, q)
    
    axes[i].imshow(config, cmap=cmap, interpolation='nearest', vmin=0, vmax=q-1)
    axes[i].set_title(f"Muestra {i+1}")
    axes[i].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 4. Análisis de distribución de colores

Verificamos que la distribución de colores sea aproximadamente uniforme.

In [ ]:
K = 10
q = 3
T = 10000
n_muestras = 100

# Acumular conteos
conteos_totales = np.zeros(q)

for i in range(n_muestras):
    config = gibbs_sampler_q_coloraciones(K, q, T, semilla=i)
    conteos_totales += contar_colores(config, q)

conteos_promedio = conteos_totales / n_muestras

print(f"Distribución promedio de colores (K={K}, q={q}, n={n_muestras}):")
for color in range(q):
    esperado = K*K / q
    print(f"  Color {color}: {conteos_promedio[color]:.2f} (esperado: {esperado:.2f})")

# Graficar
graficar_distribucion_colores(conteos_promedio, q)
plt.axhline(K*K/q, color='r', linestyle='--', label='Uniforme esperado')
plt.legend()
plt.show()

## 5. Comparación para diferentes valores de q

Analizamos cómo cambia el comportamiento con diferentes números de colores.

In [ ]:
K = 10
T = 10000
q_valores = [2, 3, 4, 5]

fig, axes = plt.subplots(2, 2, figsize=(12, 12))
axes = axes.flatten()

for idx, q in enumerate(q_valores):
    config = gibbs_sampler_q_coloraciones(K, q, T, semilla=42)
    cmap = plt.cm.get_cmap('tab10', q)
    
    axes[idx].imshow(config, cmap=cmap, interpolation='nearest', vmin=0, vmax=q-1)
    axes[idx].set_title(f"q={q}")
    axes[idx].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Análisis cuantitativo de uniformidad
K = 10
T = 10000
q_valores = [2, 3, 4, 5, 6, 7, 8, 9, 10]
n_muestras = 100

varianzas = []

for q in q_valores:
    conteos_totales = np.zeros(q)
    
    for i in range(n_muestras):
        config = gibbs_sampler_q_coloraciones(K, q, T, semilla=i)
        conteos_totales += contar_colores(config, q)
    
    conteos_promedio = conteos_totales / n_muestras
    varianza = np.var(conteos_promedio)
    varianzas.append(varianza)

print("Varianza de la distribución de colores vs q:")
for q, var in zip(q_valores, varianzas):
    print(f"  q={q:2d}: var={var:.2f}")

# Gráfico
plt.figure(figsize=(10, 6))
plt.plot(q_valores, varianzas, 'o-', linewidth=2, markersize=8)
plt.xlabel('q (número de colores)')
plt.ylabel('Varianza entre colores')
plt.title('Uniformidad de la distribución de colores')
plt.grid(True, alpha=0.3)
plt.show()